In [21]:
! pip install langchain langchain_experimental

In [22]:
# PromptTemplate
from langchain.prompts.prompt import PromptTemplate

template_string = "generate code for {text} in {language}"

prompt = PromptTemplate.from_template(template=template_string)

prompt.save('sample_template.json')
# prompt.save('sample_template.yaml')
s_prompt = PromptTemplate.from_file('sample_template.json')

In [23]:
# Chat Prompt Template
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "Hi!")
    ]
)

template.schema()

{'title': 'ChatPromptTemplate',
 'description': 'A prompt template for chat models.\n\nUse to create flexible templated prompts for chat models.\n\nExamples:\n\n    .. code-block:: python\n\n        from langchain_core.prompts import ChatPromptTemplate\n\n        template = ChatPromptTemplate.from_messages([\n            ("system", "You are a helpful AI bot. Your name is {name}."),\n            ("human", "Hello, how are you doing?"),\n            ("ai", "I\'m doing well, thanks!"),\n            ("human", "{user_input}"),\n        ])\n\n        messages = template.format_messages(\n            name="Bob",\n            user_input="What is your name?"\n        )',
 'type': 'object',
 'properties': {'name': {'title': 'Name', 'type': 'string'},
  'input_variables': {'title': 'Input Variables',
   'type': 'array',
   'items': {'type': 'string'}},
  'input_types': {'title': 'Input Types', 'type': 'object'},
  'output_parser': {'$ref': '#/definitions/BaseOutputParser'},
  'partial_variables': 

In [24]:
from dotenv import load_dotenv
load_dotenv()
! pip install openai

In [25]:
# Open AI LLM
from langchain.llms.openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo")

d:\Revature Training\Daily Workspaces\7_11thFeb\venv\Lib\site-packages\langchain_community\llms\openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
d:\Revature Training\Daily Workspaces\7_11thFeb\venv\Lib\site-packages\langchain_community\llms\openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [26]:
# Chat Model - Open AI
from langchain.chat_models import ChatOpenAI

chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")

In [32]:
# HuggingFace API
from langchain.llms.huggingface_hub import HuggingFaceHub

llm2 = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature": 1.0, "max_length": 200}
)

In [28]:
# Chat Model - Deep Infra
from langchain.llms.deepinfra import DeepInfra
from langchain_experimental.chat_models import Llama2Chat

llm3 = DeepInfra(
    model_id="meta-llama/Llama-2-70b-chat-hf"
)

llm3.model_kwargs = {
    "temperature": 0.0
}

model = Llama2Chat(llm=llm3)

In [29]:
# String Output Parser
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [33]:
# LLM Chain
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm2,
    prompt=prompt,
    output_parser=output_parser
)

chain.invoke({"text": "function to take distance in meters and return in centimeter", "language": "python"})

{'text': 'd = int(input()) if d  0: d = d / 100 if d  0: d = d / 100 * d / 100 if d  0: d = d / 100 * d / 100 if d  0: d = d / 100 * d / 100 if d',
 'language': 'python'}

In [34]:
#LLM chain
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = prompt | llm2 | parser

chain.invoke({'text':'print prime numbers', 'language':'python'})

'if x%2==0: print(x) else: print(x%3)'

In [37]:
#LLM Sequential Chain
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.runnables import RunnableParallel

company_name_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are an assistant that provides a single company name based on the description provided by the user"),
        HumanMessagePromptTemplate.from_template("{text}")

    ]
)

slogan_prompt_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="genrate a slogan from company name provided by the user and print it along with the name of the company"),
        HumanMessagePromptTemplate.from_template("{company_name}")

    ]
)


chain = RunnableParallel(
    company_name = company_name_prompt | model,
    company_slogan = {'company_name' : company_name_prompt | model} | slogan_prompt_template | model | parser 
)
# chain = {'company_name' : company_name_prompt | model} | slogan_prompt_template | model | parser 

chain.invoke({'text':'a company that sells flying cars'})

{'company_name': AIMessage(content='  Sure, based on your description, the company name that comes to mind is "Terrafugia". Terrafugia is a company that specializes in developing and manufacturing flying cars, also known as personal aerial vehicles (PAVs). They aim to create practical and safe flying cars that can be used for personal transportation and other applications.'),
 'company_slogan': '  Sure, here\'s a slogan for Terrafugia:\n\n"Terrafugia: Take to the skies, on your own terms."\n\nHere\'s a breakdown of the slogan:\n\n* "Take to the skies" refers to the ability of Terrafugia\'s flying cars to take off and land vertically, allowing users to travel by air.\n* "on your own terms" emphasizes the personal nature of the vehicles, which are designed to be practical and safe for individual use.\n\nI hope this slogan captures the essence of Terrafugia\'s mission and products! Let me know if you have any other questions or requests.'}

In [39]:
# LLM Sequential Chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

code_generation_prompt = ChatPromptTemplate.from_messages(
   [
       SystemMessage(content="You are a code assistant that generates code for users input"),
       HumanMessagePromptTemplate.from_template("generate code for {text} in {language}")

   ] 
)

time_complexity_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a code assistant that evaluates the time and space complexity of code for users input"),
        HumanMessagePromptTemplate.from_template("{code}")
    ]
)


code_summary_chain = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="you are a code assitant that prints the time complexity of the code and code provided by the user and an explanation for the code"),
        HumanMessagePromptTemplate.from_template("{code} and {time_complexity} ")
    ]
)

chain = RunnableParallel(
    code_generation = code_generation_prompt | model,
    code_complexity = RunnablePassthrough.assign(code = (code_generation_prompt | model))| time_complexity_prompt | model,
    code_comparision = {'code' : (code_generation_prompt | model), 'time_complexity': ({'code' : code_generation_prompt | model} | time_complexity_prompt | model)} | code_summary_chain | model
)


chain.invoke({'text':'print prime numbers', 'language':'python'})


{'code_generation': AIMessage(content="  Sure! Here's an example of how you could use a for loop to print all prime numbers up to a certain number in Python:\n```\nn = 30\nfor i in range(2, n+1):\n    is_prime = True\n    for j in range(2, int(i**0.5) + 1):\n        if i % j == 0:\n            is_prime = False\n            break\n    if is_prime:\n        print(i)\n```\nThis code uses a nested for loop to check whether a number is prime. The outer loop iterates from 2 to n, and the inner loop iterates from 2 to the square root of the current number being checked. If the number is divisible by any number in the inner loop, it is not prime, and the inner loop breaks. If the number is not divisible by any number in the inner loop, it is prime, and it is printed.\n\nYou can also use the Sieve of Eratosthenes algorithm to find all prime numbers up to a certain number. Here's an example of how you could implement that in Python:\n```\ndef sieve_of_eratosthenes(n):\n    is_prime = [True] * (n

In [42]:
# LLM Router chain


from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
# from transformers import pipeline

sales_prompt = ChatPromptTemplate.from_messages(

    [

    SystemMessage(content="You are a helpfull, honest sales assisnt for a telecom company called brightspeed"),
    HumanMessagePromptTemplate.from_template("{text}")

    ]
    
)

service_prompt = ChatPromptTemplate.from_messages(

    [
    SystemMessage(content="You are a helpfull, honest service assisnt for a telecom company called brightspeed guide the user to resolve the issues."),
    HumanMessagePromptTemplate.from_template("{text}")
    ]
)



router_prompt = ChatPromptTemplate.from_messages(

    [
    SystemMessage(content="Categorize the user input to sales or service along with actual text"),
    HumanMessagePromptTemplate.from_template("{text}")
    ]
)


# classifier = pipeline('zero-shot-classification', candidate_labels = ['sales', 'service'])

parser = StrOutputParser()

chain = router_prompt | model | parser

def categorize_text(text):
    if("sales" in text):
        return sales_prompt
    else:
        return service_prompt
    

router_chain = RunnablePassthrough.assign(text = (router_prompt | model | parser)) | RunnableLambda(lambda text: categorize_text(text)) | model

router_chain.invoke({'text':"What are the broadband plans available?"})

AIMessage(content="  Hello! Welcome to Brightspeed Support. I'd be happy to help you with information about our broadband plans.\n\nWe offer a variety of broadband plans to fit your needs, with different speeds and data caps to choose from. Our plans are designed to provide you with reliable and fast internet connectivity for all your online activities, from browsing and streaming to online gaming and remote work.\n\nHere are some of the broadband plans we currently offer:\n\n1. Basic: This plan offers speeds of up to 50 Mbps and a data cap of 500 GB. It's perfect for light internet users who primarily use the internet for browsing, emailing, and social media.\n2. Standard: Our most popular plan, it offers speeds of up to 100 Mbps and a data cap of 1 TB. It's ideal for households with multiple devices and moderate internet usage, including streaming and online gaming.\n3. Turbo: This plan offers speeds of up to 200 Mbps and a data cap of 2 TB. It's great for heavy internet users who en

In [43]:
# LLM Transform Chain

def to_upper(input):
    return {'text':str(input).upper()}

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="you are a helpfull assistant"),
        HumanMessagePromptTemplate.from_template("{text}, print the actual text")
    ]
    
) 

chain = RunnableLambda(lambda text: to_upper(text)) | prompt | model

chain.invoke({"text":"Its me Hi!"})

AIMessage(content='  Sure, I\'d be happy to help! The actual text you provided is:\n\n"ITS ME HI!"\n\nIs there anything else I can assist you with?')

In [46]:
# LLM Math chain

from langchain.chains import LLMMathChain

math_chain = LLMMathChain.from_llm(llm2)

math_chain.invoke({"What is 2+2?"})

{'question': {'What is 2+2?'}, 'answer': 'Answer:  4'}

In [47]:
from langchain.document_loaders import PyPDFLoader

# Example of PDF  Loader
loader = PyPDFLoader('the-velveteen-rabbit.pdf')
pages = loader.load_and_split()

In [48]:
# text spliting using character splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50,
    separator='\n'
)

documents = text_splitter.split_documents(pages)


Created a chunk of size 2107, which is longer than the specified 1000
Created a chunk of size 1625, which is longer than the specified 1000
Created a chunk of size 1857, which is longer than the specified 1000
Created a chunk of size 1859, which is longer than the specified 1000
Created a chunk of size 1862, which is longer than the specified 1000
Created a chunk of size 1276, which is longer than the specified 1000
Created a chunk of size 1376, which is longer than the specified 1000
Created a chunk of size 1984, which is longer than the specified 1000
Created a chunk of size 1767, which is longer than the specified 1000
Created a chunk of size 1945, which is longer than the specified 1000
Created a chunk of size 1599, which is longer than the specified 1000
Created a chunk of size 1423, which is longer than the specified 1000


In [ ]:
# Pinecone

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# embedding model from hf
embeddings = HuggingFaceEmbeddings(
    model_name='thenlper/gte-large'
)

index = 'index1'

result = [ Pinecone.from_documents(pages,embeddings.embed_documents, index_name =index)]


In [ ]:
! pip install pinecone

In [ ]:
# import pinecone
from langchain.vectorstores.pinecone import Pinecone

import os
from dotenv import load_dotenv
load_dotenv()

key = os.getenv('PINECONE_API_KEY')

client = Pinecone(index = "index1", text_key = key, embedding = )

In [ ]:
from langchain.vectorstores.pinecone import Pinecone
from langchain.chains import RetrievalQA



client = Pinecone(api_key=key)

index = client.Index('index1')

vectordb = Pinecone(index, embeddings.embed_query, "text")

retreiver = vectordb.as_retriever()

chain = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=retreiver)

chain.run("rabbit")

In [50]:
# Multi Query Retrival

from langchain.retrievers import MultiQueryRetriever

retreiver = MultiQueryRetriever.from_llm(
    retriever= retreiver,
    llm = llm2
    )

retreiver.get_relevant_documents("the boy")

NameError: name 'retreiver' is not defined

{'task': 'function to take distance in meters and return in centimeter',
 'language': 'python',
 'text': 'm = int(input()) c = m / 100 if m == 0: c = 0 else: c = c + 1 if c == 0: c = c + 1 if c == 0: c = c + 1 if c == 0: c = c + 1 if c == 0: c'}